In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv
import os
import numpy as np
import pandas as pd
import os.path

from sklearn.model_selection import train_test_split

# Frame to Video

In [2]:
from os.path import isfile, join
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    try:
        files.sort(key = lambda x: int(x[5:-4]))
    except:
        pass
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        
        height, width, layers = img.shape
        size = (width,height)
        #print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()


In [3]:
poses = os.listdir(r"pose_recognition_data/training data/training frames/")
for fol in ['.DS_Store', 'videos', "Andrea's Cat"]:
    try:
        poses.remove(fol)
    except:
        print('except: ', fol)

In [4]:
poses

['Adho mukha svanasana - Downward-facing Dog',
 "Ananda Balasana - Happy Baby's pose",
 'Anjaneyasana - Low Lunge',
 'Ardha Hanumanasana- Half Splits Pose',
 "Balasana - Child's pose",
 'Bhujangasana - Cobra Pose',
 'Bitilasana - cow pose',
 'Marjariasana - cat pose',
 'Phalakasana - high plank',
 'Sukhasana - Easy Pose',
 'Urdhva Mukha Svanasana - Upward-Facing Dog',
 'Uttanasana - Standing Forward Bend']

In [5]:
for pose in poses:
    #try:
    pathIn= f'./pose_recognition_data/training data/training frames/{pose}/'
    pathOut = f'./pose_recognition_data/training data/training frames/videos/{pose}.avi'
    fps = 0.25
    convert_frames_to_video(pathIn, pathOut, fps)
    '''except:
        print(pathIn)'''

## 1. Pose Detections Train and predict function

In [6]:
from utils.angles import  calc_angles_from_landmarks, ANGLE_KEYS

In [7]:
ANGLE_KEYS

['0.11/12.23/24',
 '0.11.23',
 '0.12.24',
 '11.23.-23',
 '12.24.-24',
 '11/12.23/24.-23/24',
 '12.11.13',
 '23.11.13',
 '23.11.12',
 '11.12.14',
 '24.12.14',
 '24.12.11',
 '11.13.15',
 '12.14.16',
 '11.23.25',
 '11.23.24',
 '25.23.24',
 '12.24.26',
 '12.24.23',
 '26.24.23',
 '23.25.27',
 '24.26.28',
 '28.24.16',
 '27.23.15']

In [8]:
def pose_detection(video_path=0,train=True, class_name="training_classifier_name", dimensions=2):
    mp_drawing = mp.solutions.drawing_utils # Drawing helpers
    mp_holistic = mp.solutions.holistic # Mediapipe Solutions

    cap = cv2.VideoCapture(video_path)
    # Initiate holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
                # Recolor Feed
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False        
                
                # Make Detections
                results = holistic.process(image)
                angles = calc_angles_from_landmarks(results, dictionary=False)
                # Recolor image back to BGR for rendering
                image.flags.writeable = True   
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                

                # 1. Pose Detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                        )
                # Export coordinates
                try:
                    # Extract Pose landmarks
                    pose = results.pose_landmarks.landmark
                    if dimensions == 2:
                        pose_row = list(np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in pose]).flatten())
                    
                    elif dimensions ==3:
                        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                    
                    # Concate rows
                    row = pose_row
                    row += angles
                    
                    if train==True:

                        # Create file if not present
                        if os.path.isfile(r'./data/coords.csv'):
                            pass
                        # Capture Landmarks & Export to CSV
                        else:
                            print('no data/coords.csv, make one')
                            num_coords = len(results.pose_landmarks.landmark)

                            landmarks = ['class']
                            for val in range(1, num_coords+1):
                                if dimensions == 2:
                                    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'v{}'.format(val)]

                                elif dimensions == 3:
                                    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
                            landmarks += ANGLE_KEYS
                            
                            with open(r'./data/coords.csv', mode='w', newline='') as f:
                                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                                csv_writer.writerow(landmarks)

                        # Append class name 
                        row.insert(0, class_name)

                        # Export to CSV   
                        with open(r'./data/coords.csv', mode='a', newline='') as f:
                            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                            csv_writer.writerow(row) 
                    
                    # if train is not true do the prediction
                    else:
                        # Make Detections
                        X = pd.DataFrame([row])
                        body_language_class = model.predict(X)[0]
                        body_language_prob = model.predict_proba(X)[0]
                        print(body_language_class, body_language_prob)
                        
                        # Grab ear coords
                        coords = tuple(np.multiply(
                                        np.array(
                                            (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                            results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                                    , [640,480]).astype(int))
                        
                        cv2.rectangle(image, 
                                    (coords[0], coords[1]+5), 
                                    (coords[0]+len(body_language_class)*20, coords[1]-30), 
                                    (245, 117, 16), -1)
                        cv2.putText(image, body_language_class, coords, 
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                        
                        # Get status box
                        cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
                        
                        # Display Class
                        cv2.putText(image, 'CLASS'
                                    , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                        cv2.putText(image, body_language_class.split(' ')[0]
                                    , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                        
                        # Display Probability
                        cv2.putText(image, 'PROB'
                                    , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                        cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                                    , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    print('no except')
                except:
                    print('except')
                    pass
                                
                cv2.imshow('Video', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            else:
                break
    cap.release()
    cv2.destroyAllWindows()

In [9]:
training_pose = os.listdir('.\\pose_recognition_data\\training data\\training frames\\videos')
training_pose

['Adho mukha svanasana - Downward-facing Dog.avi',
 "Ananda Balasana - Happy Baby's pose.avi",
 'Anjaneyasana - Low Lunge.avi',
 'Ardha Hanumanasana- Half Splits Pose.avi',
 "Balasana - Child's pose.avi",
 'Bhujangasana - Cobra Pose.avi',
 'Bitilasana - cow pose.avi',
 'Marjariasana - cat pose.avi',
 'Phalakasana - high plank.avi',
 'Sukhasana - Easy Pose.avi',
 'Urdhva Mukha Svanasana - Upward-Facing Dog.avi',
 'Uttanasana - Standing Forward Bend.avi']

In [10]:
for pose in training_pose:
    pose_detection(
        video_path=f'.\\yoga_gesture_detection\\pose_recognition_data\\training data\\training frames\\videos\\{pose}',
        train=True, 
        class_name=f"{pose[:-4]}")
    break

no data/coords.csv, make one
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except
no except


In [11]:
os.path.isfile(f'.\\yoga_gesture_detection\\pose_recognition_data\\training data\\training frames\\videos\\{pose}')

True

In [13]:
cap = cv2.VideoCapture('testthis')

In [14]:
video_path = f'.\\yoga_gesture_detection\\pose_recognition_data\\training data\\training frames\\videos\\{pose}'
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

cap = cv2.VideoCapture(video_path)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    print(2)
    print(cap)
    print(video_path)
    while cap.isOpened():
        print(3)
        ret, frame = cap.read()
        print(ret)
        if ret == True:
            # Recolor Feed
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False        

            # Make Detections
            results = holistic.process(image)
            angles = calc_angles_from_landmarks(results, dictionary=False)

2
<VideoCapture 00000224F56C3370>
.\yoga_gesture_detection\pose_recognition_data\training data\training frames\videos\Adho mukha svanasana - Downward-facing Dog.avi
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
True
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3

3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False


False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3


False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3


False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3


...

False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3


3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False


False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3


3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False
3
False


KeyboardInterrupt: 

In [ ]:
# Grab the training data
pose_detection(video_path=r'.\yoga_gesture_detection\pose_recognition_data\training data\training frames\videos\Adho mukha svanasana - Downward_facing Dog.avi',
train=True, class_name="Adho mukha svanasana - Downward_facing Dog")
# Train different the model
# Evaluate and select the model 
# save and load the model
# Prediction
pose_detection(video_path=r'.\yoga_gesture_detection\pose_recognition_data\training data\training recording\GMT20210323-184949_Recording_640x360.mp4',
train=False)
# Deploy

In [ ]:
pose_detection(
    video_path=r'.\yoga_gesture_detection\pose_recognition_data\training data\training frames\videos\Adho mukha svanasana - Downward_facing Dog.avi',
    train=True, 
    class_name="Adho mukha svanasana - Downward_facing Dog"
)

In [ ]:
pose

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
                
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Concate rows
            row = pose_row
            
            if train==True:

                # Append class name 
                row.insert(0, class_name)
                
                if file already exist open else create

                # Export to CSV   
                with open('coords.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()